### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/admin1/Documents/GitHub/ds22_project/data/movies.csv')

### Functions

#### 1.1 --- check_non_numeric_values

In [ ]:
def check_non_numeric_values(df, column):
    """Function takes in dataset and column. No kreturn, Printing out found non numeric values in the column."""

    # convert column to numeric data type
    numeric_col = pd.to_numeric(df[column], errors='coerce')

    # get the non-numeric values and their counts
    non_numeric_values = df[column][numeric_col.isna()].value_counts()

    # check if there are any non-numeric values
    if non_numeric_values.empty:
        print("No non numeric values in that column.")
    else:
        # create a table with non-numeric values and their counts
        non_numeric_table = pd.DataFrame({'Non-Numeric Value': non_numeric_values.index,
                                          'Count': non_numeric_values.values})

        # display the table
        print(non_numeric_table)

#### 1.2 --- get_mean_median_for_column

In [ ]:
def get_mean_median_for_column(df, col_name):
    '''
    This function takes in a pandas dataframe and the name of a column in the dataframe,
    and returns the mean and median of the numeric values in the column that are not equal to 0.
    Non-numeric values are converted to 0 before calculating the mean and median.
    
    Parameters:
    - df: pandas dataframe
    - col_name: str, name of column to be processed
    
    Returns:
    - tuple of two floats: mean and median of numeric values in the column that are not equal to 0
    '''
    # Convert non-numeric values to 0
    df[col_name] = pd.to_numeric(df[col_name], errors='coerce').fillna(0)
    
    # Get the non-zero numeric values in the column
    non_zero_vals = df[col_name][df[col_name] != 0]
    
    # Calculate the mean and median of the non-zero values
    col_mean = non_zero_vals.mean()
    col_median = non_zero_vals.median()
    
    return col_mean, col_median


#### 1.3 --- replace_missing_values

In [ ]:
def replace_missing_values(df, col_name, stat='mean'):
    '''
    This function takes in a pandas dataframe and the name of a column in the dataframe.
    It drops all rows where the value of the column is 0, and replaces those values with either 
    the mean or median of the rest of the values in the column, as specified by the user.
    It also replaces any NaN values in the column with the same statistic as the missing values.
    
    Parameters:
    - df: pandas dataframe
    - col_name: str, name of column to be processed
    - stat: str, either 'mean' or 'median', determines which statistic to use
    
    Returns:
    - df: pandas dataframe with modified column
    '''
    # Calculate the selected statistic of the non-zero/non-NaN values in the column
    if stat == 'mean':
        stat_val = np.nanmean(df[df[col_name].notnull() & (df[col_name] != 0)][col_name])
    elif stat == 'median':
        stat_val = np.nanmedian(df[df[col_name].notnull() & (df[col_name] != 0)][col_name])
    else:
        raise ValueError("stat must be either 'mean' or 'median'")
    
    # Replace the missing values (0 or NaN) with the selected statistic
    df.loc[(df[col_name] == 0) | (df[col_name].isnull()), col_name] = stat_val
    
    return df

#### 2.1 --- convert_to_usd

In [ ]:
def convert_to_usd(amount):
    amount.replace(' ', '')
    amount.replace('\xa0', '')
    if amount.startswith('$'):
        amount = amount.strip('$').replace(',', '')   # must remove commas
        return float(amount)   # convert str into float
    elif amount.startswith('€'):
        # Exchange rate for EUR to USD
        amount = amount.strip('€').replace(',', '')
        return float(amount) * 1.06 
    elif amount.startswith('¥'):
        # Exchange rate for YEN to USD
        amount = amount.strip('¥').replace(',', '')
        return float(amount) * 0.0075
    elif amount.startswith('₹'):
        # Exchange rate for RPL to USD
        amount = amount.strip('₹').replace(',', '')
        return float(amount) * 0.012 
    elif amount.startswith('SEK'):
        # Exchange rate for SEK to USD
        amount = amount.strip('SEK').replace(',', '')
        return float(amount) * 0.094
    elif amount.startswith('DKK'):
        # Exchange rate for RPL to USD
        amount = amount.strip('DKK').replace(',', '')
        return float(amount) * 0.14
    elif amount.startswith('£'):
        # Exchange rate for RPL to USD
        amount = amount.strip('£').replace(',', '')
        return float(amount) * 1.21  
    else:
        return None

#### 2.2 --- adjust_for_inflation

In [ ]:
def adjust_for_inflation(df, column_name, year_column, new_column, drop_original=True):
    data = {
        "1990": 5.398,
        "1991": 4.235,
        "1992": 3.0288,
        "1993": 2.9517,
        "1994": 2.6074,
        "1995": 2.8054,
        "1996": 2.9312,
        "1997": 2.3377,
        "1998": 1.5523,
        "1999": 2.188,
        "2000": 3.3769,
        "2001": 2.8262,
        "2002": 1.586,
        "2003": 2.2701,
        "2004": 2.6772,
        "2005": 3.3927,
        "2006": 3.2259,
        "2007": 2.8527,
        "2008": 3.8391,
        "2009": -0.3555,
        "2010": 1.64,
        "2011": 3.1568,
        "2012": 2.0693,
        "2013": 1.4648,
        "2014": 1.6222,
        "2015": 0.1186,
        "2016": 1.2616,
        "2017": 2.1301,
        "2018": 2.4426,
        "2019": 1.8122,
        "2020": 1.2336,
        "2021": 4.6979
    }
    
    # Create a new column in the DataFrame to store the adjusted values
    df[new_column] = 0

    # Loop over the rows in the DataFrame
    for index, row in df.iterrows():
        # Get the year from the row
        year = row[year_column]
        # Skip the row if the value in the specified column is NaN
        if pd.isna(row[column_name]):
            continue
        # Get the inflation rate for each year from the dictionary
        inflation_rates = [data[str(yr)] for yr in range(year, 2022)]
        # Calculate the total inflation factor by multiplying the inflation rates together
        total_inflation_factor = 1
        for rate in inflation_rates:
            total_inflation_factor *= 1 + (rate / 100)
        # Get the value from the specified column
        value = row[column_name]
        # Adjust the value for inflation using the total inflation factor
        adjusted_value = value * total_inflation_factor
        # Round the result to two decimal places and store it in the new column
        df.at[index, new_column] = round(adjusted_value, 2)

    # Drop the original column if specified
    if drop_original:
        df = df.drop(columns=[column_name])

    # Return the DataFrame with the adjusted values
    return df


#### 3.1 --- one_hot_encoding_column

In [ ]:
def one_hot_encoding_column(dataset, column, separator=", ", prefix=""):
    """
    Performs one-hot encoding on the specified column of the given dataset.
    dataset: The dataset to be processed.
    column: The name of the column to be one-hot encoded.
    separator: The separator used in the values of the specified column. Defaults to ",".
    prefix: Optional string to be added in front of each new column name. Defaults to "".
    returns: the new dataset with the specified column one-hot encoded.
    """

    # 1. Creating a list with all the values mentioned in the dataset
    value_list = [values.split(separator) for values in dataset[column]]

    # 2. Creating a set with value categories
    unique_v = {value for values in value_list for value in values}

    # 3. Performing one-hot encoding using get_dummies method
    value_subtable = pd.get_dummies(dataset[column].str.split(separator, expand=True).stack()).reset_index(level=1, drop=True)
    value_subtable = value_subtable.groupby(value_subtable.index).sum()

    # 4. Adding the prefix to the column names
    if prefix:
        value_subtable.columns = [prefix + str(col) for col in value_subtable.columns]

    # 5. Merging the subtable with the main dataset
    dataset_processed = pd.merge(dataset, value_subtable, left_index=True, right_index=True, how='left')
    dataset_processed.drop(columns=[column], inplace=True)

    # 6. Returning the new dataset
    return dataset_processed

#### 3.2 --- one_hot_coding_binary

In [ ]:
def one_hot_coding_binary(dataset, original_column, prefix, file_column, file_location, separator=", ", num_categories=1, drop_original=True):
    if num_categories not in range(1,5):
        raise ValueError("num_categories must be between 1 and 4")

    for i in range(1, num_categories+1):
        dataset[f"{prefix}_no_{i}"] = dataset[original_column].str.split(separator, expand=True)[i-1]

    dataset_categories = pd.read_csv(file_location)

    for i in range(1, num_categories+1):
        replace = dataset[f"{prefix}_no_{i}"].isin(dataset_categories[file_column])
        dataset[f"{prefix}_no_{i}_binary"] = replace.astype(int)

    if drop_original:
        dataset.drop(columns=[original_column], inplace=True)

    if num_categories == 1:
        dataset.drop(columns=[f"{prefix}_no_1"], inplace=True)
        dataset.rename(columns={f"{prefix}_no_1_binary": f"{prefix}"}, inplace=True)
    else:
        for i in range(1, num_categories+1):
            dataset.drop(columns=[f"{prefix}_no_{i}"], inplace=True)
            dataset.rename(columns={f"{prefix}_no_{i}_binary": f"{prefix}_{i}"}, inplace=True)

        if num_categories == 5:
            dataset.rename(columns={f"{prefix}_all_binary": f"{prefix}_all"}, inplace=True)

    return dataset


In [ ]:
df.info()

In [ ]:
df.head()

### 1.Title

In [ ]:
rows_pre_title = len(df)
col_pre_title = df.shape[1]

#### 1.1Unqiue/ Dupclicate values

In [ ]:
num_unique_values = df['Title'].nunique()

In [ ]:
print(num_unique_values)

In [ ]:
duplicates = df[df.duplicated(['Title'], keep=False)].sort_values(by=['Title'])

duplicates

In [ ]:
df = df.drop('Title', axis=1)

In [ ]:
#check it has been dropped

In [ ]:
df.info()

In [ ]:
rows_post_title = len(df)
col_post_title = df.shape[1]

### 2.Rating

In [ ]:
rows_pre_rating = len(df)
col_pre_rating = df.shape[1]

#### 2.1 Missing values

In [ ]:
#dropping all rows that have missing values in the column Rating
df = df.dropna(subset=['Rating'])

In [ ]:
#A quick check to see that the row was removed from the dataset
df.info()

#### 2.2 Outliers

In [ ]:
# Count how many values are between 0 and 10 (inclusive)
column_values = df['Rating']
count = column_values.between(0, 10).sum()

In [ ]:
print(count)

In [ ]:
rows_post_rating = len(df)
col_post_rating = df.shape[1]

### 3.Year

In [ ]:
rows_pre_year = len(df)
col_pre_year = df.shape[1]

#### 3.1 Outliers

In [ ]:
# Count how many values are between 0 and 10 (inclusive)
column_values = df['Year']
count = column_values.between(2002, 2023).sum()

In [ ]:
print(count)

In [ ]:
df.info()

In [ ]:
rows_post_year = len(df)
col_post_year = df.shape[1]

### 4.Month

In [ ]:
rows_pre_month = len(df)
col_pre_month = df.shape[1]

#### 4.1 Unqiue values

In [ ]:
# Check all unique values in the column Month
column_values = df['Month'].unique()
print(column_values)

In [ ]:
# Count number of values that aren't one of the 12 months
count = sum(value not in ['January', 'February', 'March', 'April', 'May', 'June', 
                          'July', 'August', 'September', 'October', 'November', 'December']
            for value in column_values)
print(count)

In [ ]:
#Drop the unwanted rows
# List of valid months
valid_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
                'August', 'September', 'October', 'November', 'December']
# Drop rows with invalid months from the original dataframe
df.drop(index=df[~df['Month'].isin(valid_months)].index, inplace=True)

In [ ]:
#Check that it has been dropped
column_values = df['Month'].unique()
print(column_values)

In [ ]:
#checking the rows have been dropped
df.info()

#### 4.5 Converting row to numeric

In [ ]:
# Define a dictionary to map months to integers
month_to_int = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

# Apply the map() method to convert the values
df['Month'] = df['Month'].map(month_to_int)

# Convert the type of the column to int
df['Month'] = df['Month'].astype(int)

In [ ]:
#checking the month-values has been rplaced by 1-12 and the column converted to int
df.info()

In [ ]:
column_values = df['Month'].unique()
column_values.sort()
print(column_values)

In [ ]:
rows_post_month = len(df)
col_post_month = df.shape[1]

### 5.Runtime

In [ ]:
rows_pre_runtime = len(df)
col_pre_runtime = df.shape[1]

In [ ]:
df.info()

##### 5.1 Check non numeric

In [ ]:
check_non_numeric_values(df, "Runtime")

##### 5.2 Drop non numeric values

In [ ]:
# Convert the 'Numbers' column to numeric values, converting non-numeric values to NaN
df['Runtime'] = pd.to_numeric(df['Runtime'], errors='coerce')
df = df.dropna(subset=['Runtime'])

In [ ]:
#checking the type has been changed
df.info()

#### 5.3 Outliers

In [ ]:
# Count how many values are between 0 and 10 (inclusive)
column_values = df['Runtime']
count = column_values.between(30, 300).sum()

In [ ]:
print(count)

In [ ]:
rows_post_runtime = len(df)
col_post_runtime = df.shape[1]

### 7.Directors

In [ ]:
rows_pre_directors = len(df)
col_pre_directors = df.shape[1]

In [ ]:
df.info()

#### 7.1 one_hot_encoding

In [ ]:
num_directors = 2
prefix_col = "top_50_director"
df = one_hot_coding_binary(df, "Directors", prefix_col, "Name", "C:/Users/admin1/Documents/GitHub/ds22_project/data/top_50_directors.csv", num_categories=num_directors, drop_original=True)


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Calculate and print sum of each new column.
sums = []
for i in range(1, num_directors+1):
    col_name = prefix_col + "_" + str(i)
    if col_name in df.columns:
        sums.append((col_name, df[col_name].sum()))

In [ ]:
df.info()

In [ ]:
rows_post_directors = len(df)
col_post_directors = df.shape[1]

### 8.Stars

In [ ]:
rows_pre_stars = len(df)
col_pre_stars = df.shape[1]

In [ ]:
df.head(2)

In [ ]:
df.info()

#### 8.1 one_hot_encoding

In [ ]:
num_stars = 4
prefix_col = "top_1000_Stars"
df = one_hot_coding_binary(df, "Stars", prefix_col, "Name", "C:/Users/admin1/Documents/GitHub/ds22_project/data/top_1000_actors.csv", num_categories=num_stars, drop_original=True)


In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
# Calculate and print sum of each new column.
sums = []
for i in range(1, num_stars+1):
    col_name = prefix_col + "_" + str(i)
    if col_name in df.columns:
        sums.append((col_name, df[col_name].sum()))
        
print(sums)

In [ ]:
rows_post_stars = len(df)
col_post_stars = df.shape[1]

### 9.Genre

In [ ]:
rows_pre_genre = len(df)
col_pre_genre = df.shape[1]

In [ ]:
df.info()

#### 9.1 - one_hot_encoding

In [ ]:
df = one_hot_encoding_column(df, "Genre", prefix = "genre_")

In [ ]:
#checking new dataset
df.info()
df.head(2)

In [ ]:
rows_post_genre = len(df)
col_post_genre = df.shape[1]

### 10.Filming_location

In [ ]:
rows_pre_filming_location = len(df)
col_pre_filming_location = df.shape[1]

In [ ]:
df.info()

In [ ]:
df.head(2)

In [ ]:
#### 10.1 Unique Values
column_values = df['Filming_location'].value_counts().sort_values(ascending=False)
print(len(column_values))
print(column_values)

In [ ]:
#Seeing 75 movies with Unknown filming_location. How can we replace them? and seeing 97 unique filming locations.
#We we´re discussing if movies mostly are beeing done w/ green screen.
#maybe remove the whole column?

#### 10.2 - Drop the column

In [ ]:
df = df.drop("Filming_location", axis=1)

In [ ]:
#check that it has been removed
df.info()

In [ ]:
rows_post_filming_location = len(df)
col_post_filming_location = df.shape[1]

### 11.Budget / 12.Income

In [ ]:
rows_pre_budget = len(df)
col_pre_budget = df.shape[1]
rows_pre_income = len(df)
col_pre_income = df.shape[1]

In [ ]:
df.head(4)

#### 1.1 convert to USD and strip of non numeric characters

In [ ]:
df['Budget'] = df['Budget'].apply(convert_to_usd)
df['Income'] = df['Income'].apply(convert_to_usd)

In [ ]:
#check the change
df.head(4)

#### 1.2 --- calc with inflation

In [ ]:
df = adjust_for_inflation(df, "Budget", "Year", "Budget_inf", drop_original=True)
df = adjust_for_inflation(df, "Income", "Year", "Income_inf", drop_original=True)

In [ ]:
df.head(4)

#### 1.3 --- Outliers

In [ ]:
#check number of rows in Budget > amount
count = ((df['Budget_inf'] > 0) & (df['Budget_inf'] < 50000)).sum()
print(count)
filtered_df = df[(df['Budget_inf'] > 0) & (df['Budget_inf'] < 50000)]
filtered_df.head()

In [ ]:
# Drop rows
df.drop(filtered_df.index, inplace=True)

In [ ]:
df.info()

In [ ]:
#check number of rows in Income < amount
count = ((df['Income_inf'] > 0) & (df['Income_inf'] < 50000)).sum()
print(count)
filtered_df = df[(df['Income_inf'] > 0) & (df['Income_inf'] < 50000)]
filtered_df.head()

In [ ]:
# drop rows
df.drop(filtered_df.index, inplace=True)

In [ ]:
df.info()

In [ ]:
#checking the change
df.head(4)

### Notice -- The function to calculate inflation has turned all NaN into 0. (Had to be done to be able to calculate)

#### 1.4 --- Missing Values

In [ ]:
#Since the missing values are 0 instead of NaN at this point (and 0 would also be treated like missing value )

# count the number of occurrences of 0 in col1
count_col1 = (df['Budget_inf'] == 0).sum()

# count the number of occurrences of 0 in col2
count_col2 = (df['Income_inf'] == 0).sum()

# count the number of occurrences of 0 in both col1 and col2
count_both = ((df['Budget_inf'] == 0) & (df['Income_inf'] == 0)).sum()

# print the results
print('Number of zeros in col1:', count_col1)
print('Number of zeros in col2:', count_col2)
print('Number of zeros in both col1 and col2:', count_both)

#### 1.5 --- Dropping rows

In [ ]:
# Drop rows where both Budget_inf and Profit_inf are 0
df = df[(df['Budget_inf'] != 0) | (df['Income_inf'] != 0)]

In [ ]:
df.info()

In [ ]:
df.head()

#### 1.6 --- Create Profit column

In [ ]:
df['Profit_inf'] = df['Income_inf'] - df['Budget_inf']
mask = (df['Income_inf'] == 0) | (df['Budget_inf'] == 0)
df.loc[mask, 'Profit_inf'] = 0

In [ ]:
df.head()

#### 1.7 --- calculate mean profit

In [ ]:
mask = df['Profit_inf'] > 0
df_filtered = df[mask]
mean_profit = df_filtered['Profit_inf'].mean()

In [ ]:
print(mean_profit)

## Problem.... movies w/ income > $218M will get a negative budget..

#### 1.8 --- Create ROI column and calculate mean_roi

In [ ]:
df['ROI_inf'] = (df['Income_inf'] - df['Budget_inf']) / df['Income_inf']
mask = (df['Income_inf'] == 0) | (df['Budget_inf'] == 0)
df.loc[mask, 'ROI_inf'] = 0

In [ ]:
#calc mean & median roi of the rows w/ values in both
mask = df['ROI_inf'] != 0
df_filtered = df[mask]
mean_roi = df_filtered['ROI_inf'].mean()
median_roi = df_filtered['ROI_inf'].median()


In [ ]:
print(mean_roi)

In [ ]:
print(median_roi)

In [ ]:
df.to_csv('data.csv', index=False)


In [ ]:
#why is mean roi negativ 200%? we´ll because some movies failed big and have big negative ROI

In [ ]:
df.head()

#### 1.9 -- replace 0 values in ROI_inf to median_ROI_inf

In [ ]:
df['ROI_inf'] = df['ROI_inf'].replace(0, median_roi)

In [ ]:
df.head()

In [ ]:
# replace 0 and NaN values in the income_inf/budget_inf column with the median_roi
df.loc[df['Income_inf'].isna() | (df['Income_inf'] == 0), 'Income_inf'] = df['Budget_inf'] * (1 + median_roi)
df.loc[df['Budget_inf'].isna() | (df['Budget_inf'] == 0), 'Budget_inf'] = df['Income_inf'] / (1 + median_roi)


In [ ]:
df.head()

#### 1.10 --- update profit_inf

In [ ]:
df['Profit_inf'] = df['Income_inf'] - df['Budget_inf']

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
rows_post_budget = len(df)
col_post_budget = df.shape[1]
rows_post_income = len(df)
col_post_income = df.shape[1]

### 13.Country_of_origin

In [ ]:
rows_pre_country_of_origin = len(df)
col_pre_country_of_origin = df.shape[1]

In [ ]:
df.info()

In [ ]:
df.head(2)

#### 13.1 --- Unique Values

In [ ]:
unique_values_count = df['Country_of_origin'].str.split(', ').explode().value_counts()
print(len(unique_values_count))
print(unique_values_count)

#### 13.2 Drop column

In [ ]:
df = df.drop('Country_of_origin', axis=1)

In [ ]:
#check it has been dropped
df.info()

In [ ]:
rows_post_country_of_origin = len(df)
col_post_country_of_origin = df.shape[1]

## Certificate

In [ ]:
rows_pre_certificate = len(df)
col_pre_certificate = df.shape[1]

In [ ]:
df.info()

#### 1 --- Unique Values

In [ ]:
unique_values = df['Certificate'].unique()

In [ ]:
unique_values_count = df['Certificate'].value_counts()

In [ ]:
print(unique_values)

In [ ]:
print(unique_values_count)

#### 1.2 --- Drop rows

In [ ]:
df = df.drop(df[(df['Certificate'].isnull()) | (df['Certificate'] == 'Not Rated') | (df['Certificate'] == 'Unrated')].index)

In [ ]:
df.info()

In [ ]:
unique_values_count_after_drop = df['Certificate'].value_counts()
print(unique_values_count_after_drop)

#### 1.3 --- one hot encoding

In [ ]:
df = one_hot_encoding_column(df, "Certificate", prefix = "rated_")

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
rows_post_certificate = len(df)
col_post_certificate = df.shape[1]

In [ ]:
#for display-porpuses at the end
#headers = ['title', 'rating', 'year', 'month', 'runtime', 'directors', 'stars', 'genre', ]
#rows_dropped = [0] * len(headers)

#for i in range(len(headers)):
#    pre_count = globals().get(f"rows_pre_{headers[i]}", 0)
#    post_count = globals().get(f"rows_post_{headers[i]}", 0)
#    rows_dropped[i] = pre_count - post_count
#    print(f"Rows dropped for {headers[i]}: {rows_dropped[i]}")

In [ ]:
#df.to_csv('movies_processed.csv', index=False)